# Telegram相关

In [ ]:
# 将用于telegram查询群组的关键词从csv提取到json
import csv

results={}
results["all"]=[]
regex_value=""
with open("dark_keywords_telegram_search/dark_keywords.csv",) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        results.setdefault(row["category"],[])
        results[row["category"]].append(row["keyword"])
        results["all"].append(row["keyword"])
        regex_value+=(row["keyword"]+"|")
        # print(regex_value)
csvfile.close()
print(regex_value[:-1])
results["regex"]=regex_value[:-1]

results

import json

with open("dark_keywords_telegram_search/dark_keywords.json","w") as jsonfile:
    json.dump(results,jsonfile,ensure_ascii=False)
jsonfile.close()

In [ ]:
# 将查询出来的group_channle_url汇总成regex表达式
import csv
results={}
regex_value=""
with open("dark_keywords_telegram_search/keyword_group_channel_url.csv",) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        regex_value+=(row["group_channel_url"]+"|")
        # print(regex_value)
csvfile.close()
print(regex_value[:-1])
results["regex"]=regex_value[:-1]

import json

with open("dark_keywords_telegram_search/keyword_group_channel_url.json","w") as jsonfile:
    json.dump(results,jsonfile,ensure_ascii=False)
jsonfile.close()

In [ ]:
# 将筛选出的282条telegram聊天记录进行整合成origin_data的格式：
# background,kind,input,content
# input_template: 下面是一段telegram的聊天记录，请判断其是否与黑灰产相关，若相关请将其进一步分类：\nrow["message_text"]
# content_template: 正常：这是一段正常的聊天记录，与黑灰产无关。其他：这是一段与黑灰产相关的聊天记录。 黑灰产：这是一段与row["kind"]相关的黑灰产聊天记录。
import csv
import re

def clean_text(text):  
    # 定义一个正则表达式模式，匹配中文、数字、英文字符、常见标点符号和换行符  
    pattern = re.compile(r'[\u4e00-\u9fa5a-zA-Z0-9，。！？、；：“”（）、。《》【】、\#\$\%\^\&\*\(\)\_\+\-\=\{\}\|\[\]\\\:\'\"\<\>\?\~\.\,\!\@\n\r\t/]+')  
      
    # 使用正则表达式查找所有匹配的字符，并将它们连接起来  
    cleaned_text = pattern.findall(text)  
      
    # 由于findall返回的是列表，每个元素是匹配的一个片段  
    # 我们需要将这些片段连接起来，形成一个完整的字符串  
    cleaned_text = ''.join(cleaned_text)  
      
    return cleaned_text 

results=[["from","kind","input","content"]]
with open("/home/w1nd/darkword/1darkword/data_crawl/darkword_data_telegram/telegram_chat_data_ready_to_merge.csv","r",encoding="utf-8") as file:
    reader = csv.DictReader(file)
    for row in reader:
        input = "下面是一段telegram的聊天记录，请判断其是否与黑灰产相关，若相关请将其进一步分类：\n"+clean_text(row["message_text"])
        # print(input)
        kind = row["kind"]
        if kind == "正常":
            content = "这是一段正常的聊天记录，与黑灰产无关。"
        elif kind =="其他":
            content = "这是一段与黑灰产相关的聊天记录。"
        else:
            content = "这是一段与"+row["kind"]+"相关的黑灰产聊天记录。"
        
        one_result=["Telegram_Chat_Message",row["kind"],input,content]
        results.append(one_result)
file.close()

with open("/home/w1nd/darkword/1darkword/data_crawl/darkword_data_telegram/telegram_chat_data_final.csv","wt",encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(results)
file.close()

# 将原始数据集随机切割成train和eval数据集（9:1）

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("/home/w1nd/darkword/1darkword/data_crawl/darkdata_all_origin.csv",encoding="utf-8")
train_df, eval_df = train_test_split(df,test_size=0.1,random_state=51)
train_file_path="/home/w1nd/darkword/1darkword/data_crawl/data_train_or_eval/darkword_train_data.csv"
eval_file_path="/home/w1nd/darkword/1darkword/data_crawl/data_train_or_eval/darkword_eval_data.csv"
train_df.to_csv(train_file_path,index=False)
eval_df.to_csv(eval_file_path,index=False)

# Atom相关

In [ ]:
# 将整理的用于大模型微调的黑话origin数据集整理成一列：csv->csv
# 有train和eval两个
import csv

train_file_path="/home/w1nd/darkword/1darkword/data_crawl/data_train_or_eval/darkword_train_data.csv"
eval_file_path="/home/w1nd/darkword/1darkword/data_crawl/data_train_or_eval/darkword_eval_data.csv"
atom_train_file_path="/home/w1nd/darkword/1darkword/data_crawl/darkword_data_atom/darkword_train_data.csv"
atom_eval_file_path="/home/w1nd/darkword/1darkword/data_crawl/darkword_data_atom/darkword_eval_data.csv"

train_results=[["text"]]
eval_results=[["text"]]
with open(train_file_path,"r",encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        text="<s>Human: "+row["input"]+"\n</s><s>Assistant: "+row["content"]+"\n</s>"
        print(text)
        train_results.append([text])
csvfile.close()
with open(eval_file_path,"r",encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        text="<s>Human: "+row["input"]+"\n</s><s>Assistant: "+row["content"]+"\n</s>"
        print(text)
        eval_results.append([text])
csvfile.close()


with open(atom_train_file_path,"wt",encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(train_results)
file.close()
with open(atom_eval_file_path,"wt",encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(eval_results)
file.close()

# Chatglm3相关

In [7]:
# 第一版测试，将所有conversation都存入一个数组[]，再写入json文件
# chatglm3-6b数据集处理
# 格式：{"conversations":[{"role":"user","content":row["input"]},{"role":"assistant","content":row["content"]},{},{}]}
import csv,json

origin_train_data_file="/home/w1nd/darkword/1darkword/data_crawl/data_train_or_eval/darkword_train_data.csv"
origin_eval_data_file="/home/w1nd/darkword/1darkword/data_crawl/data_train_or_eval/darkword_eval_data.csv"

output_data_dir="darkword_data_chatglm3/"
train_data_file=output_data_dir+"train.json"
eval_data_file=output_data_dir+"dev.json"
train_results=[]
eval_results=[]
with open(origin_train_data_file,"r",encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        sample={"conversations":[{"role":"user","content":row["input"]},{"role":"assistant","content":row["content"]}]}
        train_results.append(sample)
csvfile.close()
with open(origin_eval_data_file,"r",encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        sample={"conversations":[{"role":"user","content":row["input"]},{"role":"assistant","content":row["content"]}]}
        eval_results.append(sample)
csvfile.close()

with open(train_data_file,"wt",encoding="utf-8") as output:
    json.dump(train_results,output,ensure_ascii=False,indent=4)
output.close()
with open(eval_data_file,"wt",encoding="utf-8") as output:
    json.dump(eval_results,output,ensure_ascii=False,indent=4)
output.close()

In [2]:
# 第二版测试，将每个conversation一行一行的存入json文件中。（不知道哪个格式是对的。。）
# chatglm3-6b数据集处理
# 格式：{"conversations":[{"role":"user","content":row["content"]},{"role":"assistant","content":row["summary"]},{},{}]}
import csv,json

origin_train_data_file="/home/w1nd/darkword/1darkword/data_crawl/data_train_or_eval/darkword_train_data.csv"
origin_eval_data_file="/home/w1nd/darkword/1darkword/data_crawl/data_train_or_eval/darkword_eval_data.csv"

output_data_dir="darkword_data_chatglm3/"
train_data_file=output_data_dir+"train.json"
eval_data_file=output_data_dir+"dev.json"
results=[]

with open(origin_train_data_file,"r",encoding="utf-8") as csvfile:
    with open(train_data_file,"wt",encoding="utf-8") as outputfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            sample={"conversations":[{"role":"user","content":row["input"]},{"role":"assistant","content":row["content"]}]}
            outputfile.write(json.dumps(sample,ensure_ascii=False)+"\n")
    outputfile.close()
csvfile.close()

with open(origin_eval_data_file,"r",encoding="utf-8") as csvfile:
    with open(eval_data_file,"wt",encoding="utf-8") as outputfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            sample={"conversations":[{"role":"user","content":row["input"]},{"role":"assistant","content":row["content"]}]}
            outputfile.write(json.dumps(sample,ensure_ascii=False)+"\n")
    outputfile.close()
csvfile.close()


# Baichuan2相关

In [2]:
# 将原始数据集转换为baichun2微调的格式
# 格式：[{"id":*,"conversations":[{"from":"*","value":"*"},{"from":"gpt","value":"*"},{},{}]},{},{}]
import csv,json

train_file_path="/home/w1nd/darkword/1darkword/data_crawl/data_train_or_eval/darkword_train_data.csv"
output_file_path="/home/w1nd/darkword/1darkword/data_crawl/darkword_data_baichuan2/train_data.json"
train_results=[]

with open(train_file_path,"r",encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for index,row in enumerate(reader,start=1):
        conversation=[{"from":"human","value":row["input"]+"\n"},{"from":"gpt","value":row["content"]+"\n"}]
        one_result={"id":index,"conversations":conversation}
        train_results.append(one_result)
csvfile.close()

with open(output_file_path,"wt",encoding="utf-8") as output:
    json.dump(train_results,output,ensure_ascii=False,indent=4)
output.close()